In [2]:
from diffusers.utils import load_image, make_image_grid
from PIL import Image
import cv2
import numpy as np

original_image = load_image(
    "./dataset/structure_with_cracks0.png"
)

image = np.array(original_image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

C:\Users\dan_d\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16, use_safetensors=True)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, use_safetensors=True
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")
pipe.enable_model_cpu_offload()

Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.74it/s]


In [4]:
for i in range(10):
    output = pipe(
        "Concrete with cracks", image=canny_image
        ).images[0]
    make_image_grid([original_image, canny_image, output], rows=1, cols=3)

    output.save("controlnet"+str(i)+".jpg")

  0%|          | 0/50 [00:00<?, ?it/s]C:\Users\dan_d\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\diffusers\models\attention_processor.py:1279: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  hidden_states = F.scaled_dot_product_attention(
 48%|████▊     | 24/50 [07:23<08:00, 18.49s/it]


KeyboardInterrupt: 

In [7]:
chat_image = load_image(
    "./chatgpt_prompt_images/chatprompt2.png"
)

image = np.array(chat_image)
image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

In [8]:
for i in range(10):
    output = pipe(
        "Generate a realistic image of a concrete building with visible signs of damage, such as spalling, cracks, and weathering.", image=canny_image
        ).images[0]
    make_image_grid([original_image, canny_image, output], rows=1, cols=3)

    output.save("chat_controlnet"+str(i)+".jpg")

100%|██████████| 50/50 [14:16<00:00, 17.14s/it]
